In [2]:
import pandas as pd
import numpy as np
import requests
import json
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()

True

In [5]:
fmp_api = os.getenv('FMP_API')

In [56]:
ticker = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-price-full/AAPL?serietype=line&apikey={fmp_api}").content)['historical']

In [57]:
data = pd.DataFrame(ticker).set_index('date')[::-1]

In [58]:
data.head()

,close
date,
1980-12-12,0.128348
1980-12-15,0.121652
1980-12-16,0.112723
1980-12-17,0.115513
1980-12-18,0.118862


In [59]:
data['middle_band'] = data[['close']].rolling(window=20).mean()

In [60]:
data['20_day_stdev'] = data[['close']].rolling(window=20).std()

In [61]:
data

,close,middle_band,20_day_stdev
date,,,
1980-12-12,0.128348,NaN,NaN
1980-12-15,0.121652,NaN,NaN
1980-12-16,0.112723,NaN,NaN
1980-12-17,0.115513,NaN,NaN
1980-12-18,0.118862,NaN,NaN
...,...,...,...
2021-11-18,157.869995,150.498502,2.267825
2021-11-19,160.550003,151.091502,3.149324
2021-11-22,161.020004,151.710503,3.792990


In [62]:
data['upper_band'] = data['middle_band']+data['20_day_stdev']

In [64]:
data['lower_band'] = data['middle_band'] - data['20_day_stdev']

In [65]:
data.head(20)

,close,middle_band,20_day_stdev,upper_band,lower_band
date,,,,,
1980-12-12,0.128348,NaN,NaN,NaN,NaN
1980-12-15,0.121652,NaN,NaN,NaN,NaN
1980-12-16,0.112723,NaN,NaN,NaN,NaN
1980-12-17,0.115513,NaN,NaN,NaN,NaN
1980-12-18,0.118862,NaN,NaN,NaN,NaN
1980-12-19,0.126116,NaN,NaN,NaN,NaN
1980-12-22,0.132254,NaN,NaN,NaN,NaN
1980-12-23,0.137835,NaN,NaN,NaN,NaN
1980-12-24,0.145089,NaN,NaN,NaN,NaN


In [365]:
class TechnicalIndicators:
    
    def __init__(self, ticker):
        
        self.ticker = ticker
    
    def create_price_df(self):
        ticker = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-price-full/{self.ticker}?serietype=line&apikey={fmp_api}").content)['historical']
        data = pd.DataFrame(ticker).set_index('date')[::-1]
        
        return data
        
    def bollinger_bands(self, dataframe,period=20):
        data = dataframe.copy()
        data['middle_band'] = data[['close']].rolling(window=period).mean()
        data[str(period)+'_day_stdev'] = data[['close']].rolling(window=period).std()
        data['upper_band'] = data['middle_band']+2*data[str(period)+'_day_stdev']
        data['lower_band'] = data['middle_band'] - 2*data[str(period)+'_day_stdev']
        data['spread'] = data['upper_band'] - data['lower_band']
        data['lag_spread'] = data['spread'].shift(1)
        data['Signal'] = np.where(data['spread']>data['lag_spread'], 1.0, -1.0)
        data = data.dropna()
        
        return data
    
    def dema(self, dataframe, period1=10, period2=20):
        data = dataframe.copy()
        data[str(period1)+'ema1'] = dataframe[['close']].ewm(span=period1, adjust=False).mean()
        data[str(period1)+'ema2'] = data[str(period1)+'ema1'].ewm(span=period1, adjust=False).mean()
        data[str(period1)+'dema'] = 2*data[str(period1)+'ema1'] - data[str(period1)+'ema2']
        data[str(period2)+'ema1'] = data[['close']].ewm(span=period2, adjust=False).mean()
        data[str(period2)+'ema2'] = data[str(period2)+'ema1'].ewm(span=period2, adjust=False).mean()
        data[str(period2)+'dema'] = 2*data[str(period2)+'ema1'] - data[str(period2)+'ema2']
        data['spread'] = data[str(period1)+'dema'] - data[str(period2)+'dema']
        data['Signal'] = np.where(data[str(period1)+'dema'] > data[str(period2)+'dema'], 1.0, 0.0)
        data['Entry_Exit'] = data['Signal'].diff()
        data = data.dropna()
        return data
    
    def price_momentum(self, dataframe, smoothing1=0.0571, smoothing2=0.1, periods1=35, periods2=20):
        data = dataframe.copy()
        data['returns'] = data['close'].pct_change()
        data['smoothing_factor'] = smoothing1
        data[str(periods1)+"average"] = data['returns'].rolling(window=periods1).mean()
        smoothing_factor_list = [data.iloc[periods1][str(periods1)+"average"]]
        data = data.dropna()
        i=1
        j=0
        while i < len(data[str(periods1)+"average"]):
            smoothing_factor = data.iloc[i]['returns']*data.iloc[i]['smoothing_factor'] + smoothing_factor_list[j]*(1-data.iloc[i]['smoothing_factor'])
            smoothing_factor_list.append(smoothing_factor)
            j+=1
            i+=1
        data['35d_custom_smoothing'] = smoothing_factor_list
        data['35d_custom_10'] = data['35d_custom_smoothing']*10
        data['smoothing_factor2'] = smoothing2
        data[str(periods2)+"average"] = data['35d_custom_10'].rolling(window=periods2).mean()
        data = data.dropna()
        smoothing_factor_list2 = [data.iloc[0][str(periods2)+"average"]]
        i=1
        j=0
        while i < len(data[str(periods2)+"average"]):
            smoothing_factor = (data.iloc[i]['35d_custom_10'] - smoothing_factor_list2[j])*data.iloc[i]['smoothing_factor2'] + smoothing_factor_list2[j]
            smoothing_factor_list2.append(smoothing_factor)
            j+=1
            i+=1
        data[str(periods2)+'d_custom_smoothing'] = smoothing_factor_list2
        data['signal'] = np.where(data[str(periods2)+'d_custom_smoothing'] > data[str(periods2)+'d_custom_smoothing'].shift(1), 1.0, 0.0)
        
        return data

In [366]:
appl_indicators = TechnicalIndicators('AAPL')

In [367]:
aapl_df = appl_indicators.create_price_df()

In [368]:
appl_bb = appl_indicators.price_momentum(aapl_df)

In [369]:
appl_bb

,close,returns,smoothing_factor,35average,35d_custom_smoothing,35d_custom_10,smoothing_factor2,20average,20d_custom_smoothing,signal
date,,,,,,,,,,
1981-03-03,0.117188,-0.014084,0.0571,-0.004763,-0.000450,-0.004500,0.1,-0.019458,-0.019458,0.0
1981-03-04,0.116071,-0.009532,0.0571,-0.004019,-0.000969,-0.009686,0.1,-0.019764,-0.018481,1.0
1981-03-05,0.115513,-0.004807,0.0571,-0.004273,-0.001188,-0.011878,0.1,-0.021224,-0.017821,1.0
1981-03-06,0.114397,-0.009661,0.0571,-0.005132,-0.001672,-0.016716,0.1,-0.022876,-0.017710,1.0
1981-03-09,0.105469,-0.078044,0.0571,-0.007133,-0.006032,-0.060325,0.1,-0.026787,-0.021972,0.0
...,...,...,...,...,...,...,...,...,...,...
2021-11-18,157.869995,0.028536,0.0571,0.003197,0.003507,0.035074,0.1,0.013225,0.012212,1.0
2021-11-19,160.550003,0.016976,0.0571,0.003450,0.004276,0.042764,0.1,0.014706,0.015267,1.0
2021-11-22,161.020004,0.002927,0.0571,0.004237,0.004199,0.041994,0.1,0.016196,0.017940,1.0


In [305]:
len(appl_bb['35average'])

10292